In [1]:
import pandas as pd
import re
import os

In [56]:
import mysql.connector
db = mysql.connector.connect(user='root', password='password', database='folklore_db2')
cur = db.cursor()

In [3]:
from numpy import nan
from collections import namedtuple
import time
from IPython.display import clear_output

from pymystem3 import Mystem
m = Mystem()

In [4]:
filelist = os.listdir('./txt/')

In [6]:
def split_text(text):
    result = re.findall('^(.*?Ключевые.*?\n[\nА-Яа-я0-9\- ;]+?\n)(.*?)$', text, re.DOTALL)
    return result

In [7]:
def find_keywords(meta):
    result = [i.strip().capitalize() for i in re.findall('Ключевые слова:(.*?)\n', meta)[0].split(',')]
    if len(result) == 1 and len(result[0])<2:
        return None
    else:
        return result

In [8]:
find_keywords('Календарный обряд. Ильин день. Духов день. Троица \n32:10-32:54\nКлючевые слова: календарь, купание, запрет, Ильин день, Духов день, Троица\nСРВ-1935-МН-2018')

['Календарь', 'Купание', 'Запрет', 'Ильин день', 'Духов день', 'Троица']

In [9]:
cur.execute('SELECT id, word FROM keywords')
d_key = {}
for i in cur.fetchall():
    d_key[i[1]] = i[0]

In [13]:
def read_text(path):
    try:
        with open(path, encoding='utf-8') as f:
            return f.read()
    except:
        with open(path, encoding='cp1251') as f:
            return f.read()

In [43]:
folders = [
    '/home/dkbrz/Downloads/НАДЯ ФАЙЛЫ(с кодами исполнителей)-20191117T144444Z-001',
    '/home/dkbrz/Downloads/Настя файлы(с кодами исполнителей)-20191117T144509Z-001',
    '/home/dkbrz/Downloads/Аня файлы(с кодами исполнителей)-20191117T144709Z-001',
    '/home/dkbrz/Downloads/Маша файлы(с кодами исполнителей)-20191117T144541Z-001'
]

In [73]:
mapping = {}

for folder in folders:
    for root, _, files in os.walk(folder):
        for f in files:
            text = read_text(os.path.join(root, f))
            result = split_text(text)
            try:
                result = find_keywords(result[0][0])
                if result is None:
                    print(f)
                else:
                    mapping[f] = result
            except:
                print('!', f)
                print(result)

Архив ВШЭ – ФЭ – 37 – 20 – 2018 – 40.txt
Архив ВШЭ – ФЭ – 37 – 20 – 2018 – 43.txt
Архив ВШЭ – ФЭ – 37 – 20 – 2018 – 67.txt
Архив ВШЭ – ФЭ – 37 – 20 – 2018 – 36.txt
Архив ВШЭ – ФЭ – 37 – 20 – 2018 – 26.txt
Архив ВШЭ – ФЭ – 37 – 20 – 2018 – 69.txt
Архив ВШЭ – ФЭ – 37 – 20 – 2018 – 45.txt
! Архив ВШЭ – Э – 37 – 20 – 2018 – 434.txt
[]


In [74]:
mapping['Архив ВШЭ – Э – 37 – 20 – 2018 – 434.txt'] = [
    'календарь', 'купание', 'запрет', 'Ильин день', 'Духов день', 'Троица'
]

In [75]:
def clean_name(name):
    return name.replace('.txt', '')

In [76]:
new_name_mapping = {
    clean_name(key): value
    for key, value in mapping.items()
}

In [77]:
cur.execute('SELECT old_id, id FROM texts')
texts_name_to_id = {key: value for key, value in cur.fetchall()}

In [78]:
for key in new_name_mapping:
    if key not in texts_name_to_id:
        print('-', key)
#     else:
#         print('+', key)

In [112]:
misspell = {"Благословение":"Благословлять","Быличка":"Былички","Вербное воскресенье":"Вербное воскресение","Воскресенье":"Воскресенье (день недели)","Гад":"Гады","Гадание":"Гадания","Гуляние":"Гулять","Гулянь":"Гулять","Духовные стихи.":"Духовные стихи","Жаворонки":"Жаворонок","Жена":"Супруги","Заговор":"Заговоры","Запрет":"Запреты","Знахари":"Знахарь","Иван купала":"Иван Купала","Иконы":"Икона","Каравай":"Коровай","Кикиимора":"Кикимора","Кладище":"Кладбище","Колдуны":"Колдун","Красный":"Красный цвет","Крещение":"Крещенье (праздник)","Крещенье":"Крещенье (праздник)","Легенда":"Легенды","Музыкальный инструмент":"Музыкальные инструменты","Мыть":"Мытье","Новороденный":"Новорожденный","Оберег":"Обереги","Обмывание":"Обмывание покойника","Осыпать":"Осыпание","Песня":"Песни","Петр и павел":"Петр и Павел","Петра и павла день":"Петра и Павла день","Петух 31:07-36:26":"Петух","Покойники":"Покойник","Покров":"Покров (календ.)","Поминальный дни":"Поминальные дни","Праздник":"Праздники","Предание":"Предания","Предание топоним.":"Предания топоним.","Предания топоним":"Предания топоним.","Приговор":"Приговоры","Примета":"Приметы","Прозвища":"Прозвище","Птица":"Птицы","Радоница":"Радуница","Рождество богородицы":"Рождество Богородицы","Ряжение":"Ряженье","Свечаэ=":"Свеча","Святой":"Святые","Смерть":"Жизнь-смерть","Сновидение":"Сновидения","Соседи":"Сосед","Танцевать":"Танец","Угощать":"Угощение","Урожай":"Урожай-неурожай","Успение богородицы":"Успение Богородицы","Шутка":"Шутки","Яйца":"Яйцо", "Бывальщина":"Былички","Егорьев день":"Юрьев день","Егорьер день":"Юрьев день","Застолье":"Трапеза ритуальная","Знамение":"Знак-знамение","Игры":"Игры фольк.","Коса":"Коса (волосы)","Лук":"Лук (раст.)","Мальчишник":"Девичник","Молодые":"Молодожены","Муж":"Супруги","Обрядовое деревце":"Деревце обрядовое","Петровский обряд":"Петра и Павла день","Плач":"Причитания","Поминание":"Поминальные дни","хороны":"Похоронный обряд","Рассказ о прошлом":"Устная история","Ритуальная речь":"Говорить","Родственники":"Родство","Ряженые":"Ряженье","Ряженый":"Ряженье","Свадебный":"Свадебный обряд","Свадьба":"Свадебный обряд","Сваты":["Сват", "Сваха"], "Гора, лес":["Гора", "Лес"], "Семья":["Родство","Супруги"],"Стихи":"Народная литература","Частушка":"Песни", "Молодой":"Молодой-старый", 'Похороны':'Похоронный обряд', 'Гора лес':['Гора', 'Лес'], 'Осыпать': 'Осыпание'}

In [113]:
d_key_low = {key.lower(): value for key, value in d_key.items()}

In [114]:
def get_part(text_name, mapping_res):
    if mapping_res is None:
        return []
    res = []
    for kw in mapping_res:
        if kw.lower() in d_key_low:
            res.append([d_key_low[kw.lower()], texts_name_to_id[text_name]])
        elif kw in misspell:
            r = misspell[kw]
            if type(r) == list:
                for w in r:
                    res.append([d_key_low[w.lower()], texts_name_to_id[text_name]])
            else:
                #try:
                    res.append([d_key_low[r.lower()], texts_name_to_id[text_name]])
                #exx
    return res

In [115]:
get_part('Архив ВШЭ – Э – 37 – 20 – 2018 – 434', ['Гора лес'])

[[181, 4142], [487, 4142]]

In [116]:
data = []
for key in new_name_mapping:
    res = get_part(key, new_name_mapping[key])
    data.extend(res)

In [118]:
len(data)

2660

In [120]:
cur.executemany('INSERT t_k (id_keyword, id_text) VALUES (%s, %s)', data)

In [121]:
db.commit()